# 경쟁사 신상품 이메일 알림 (업종 - 브랜드 하나 잡아서!)

제품에 대한 경쟁우위를 갖기 위해서는,
경쟁사 제품에 대한 이해도가 필수적입니다. 

이 파트에서는 경쟁사의 신상품에 대해 알람을 설정함으로써,
변화하는 경쟁상황에 대해 발빠르게 대처할 수 있는 발판을 마련합니다.

## 설계

### 준비물: (경쟁사 정하기(URL), 카테고리 u, 상품 페이지)

- 여기서는 마약베개로 유명한 '바디럽'을 경쟁사로 하겠습니다.(https://bodyluv.kr/index.html)


0. 브라우저를 세팅한다.
1. 경쟁사 사이트에 들어간다.
2. 카테고리 url을 정리하고 접속한다.
3. 리스트를 읽고 형식에 맞게 저장.
4. 기존 파일을 불러와서 변경사항 확인
5. 새로운 파일 저장
6. 변경사항이 있으면 메일을 보낸다.

#### 0. 브라우저를 세팅한다.

In [36]:
# 브라우저 설정

from selenium import webdriver
from pprint import pprint


# TODO: Option에 대한 설명 필요.
# options = None
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": '.'
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.implicitly_wait(30)


#### 1. 경쟁사 사이트에 들어간다.

In [37]:
# target_url = bodyluv_url
bodyluv_url = 'https://bodyluv.kr/index.html'

driver.get(bodyluv_url)

#### 2. 카테고리 url을 정리하고 접속한다.

In [38]:
# url에 접속하는 두가지 방법.
# 1. selenium action chain으로 클릭을 인식시킨다.
#### HTML 소스코드(markup) 종속적
all_products = driver.find_element_by_xpath('//*[@id="main"]/div[4]/div/ul/li[6]/a')

actions = webdriver.ActionChains(driver)
actions.move_to_element(all_products).click()
actions.perform()

# 또는..
# 2. html을 분석해서 <a>태그 (anchor)의 href(hyper reference)를 파악한다.
#### url 구조(backend) 종속적
driver.get('https://bodyluv.kr/product/list.html?cate_no=24')

### 위 두종류의 예제는 같은 결과(브라우저 링크를 이동하는)를 보여줍니다.


#### 3. 리스트를 읽고 형식에 맞게 저장한다.

In [43]:
# 코드 설명 보충 위한 페이지 재로드
driver.get('https://bodyluv.kr/product/list.html?cate_no=24')

# full_list tag: .prdList
bodyluv_list = []

# 아래처럼 읽으면 됩니다. 하지만, Pagination이 있으니 끝까지 다 읽어야겠지요?
# 반복문(loop)을 사용해야 합니다.


# 반복문 적용 X
# prd_list = driver.find_element_by_class_name('prdList')
# prd_items = prd_list.find_elements_by_xpath('//li[@itemprop="itemListElement"]')
# bodyluv_list += prd_items


# # 반복문 적용 O 
last_page_href = 'none'

next_link = None

while(True):
    prd_list = driver.find_element_by_class_name('prdList')
    prd_items = prd_list.find_elements_by_xpath('//li[@itemprop="itemListElement"]')

    # 형식에 맞게 저장.
    # 확실히 코드가 길어지니 복잡하죠?
    # 아래에 function으로 바꿔 보겠습니다.
    for item in prd_items:
        thumbnail_box = item.find_element_by_class_name('thumbnail')
        desc_box = item.find_element_by_class_name('description')
    
        bodyluv_list.append({
            'link': thumbnail_box.find_element_by_tag_name('a').get_attribute('href'),
            'thumbnail': thumbnail_box.find_element_by_tag_name('img').get_attribute('src'),
            'title': desc_box.find_element_by_class_name('name')\
                    .find_element_by_class_name('gtm-impression-click-target').text,
            'origin_price':desc_box.find_element_by_xpath('//ul/li[1]/span[1]'),
            'sales_price': desc_box.find_element_by_xpath('//ul/li[2]/span[1]'),
        })
    
    next_page = driver.find_element_by_xpath('//*[@id="contents"]/div[3]/a[3]')
    next_link = next_page.get_attribute('href')
    
    # 형식에 맞게 정리 

    if (next_link.split('#')[-1]==last_page_href):
        break
    driver.get(next_link)

# print(len(bodyluv_list), bodyluv_list)







[{'link': 'https://bodyluv.kr/product/%EB%B0%94%EB%94%94%EB%9F%BD-%EC%A0%9C%EB%A1%9C%EB%8D%94%EC%8A%A4%ED%8A%B8-%EB%B0%B0%EA%B8%B0%EB%A7%88%EC%8A%A4%ED%81%AC-%EC%86%8C%ED%98%95%EB%8C%80%ED%98%95-5%EB%A7%A4/48/category/24/display/1/', 'thumbnail': 'https://bodyluv.kr/web/product/big/201711/48_shop1_919569.jpg', 'title': '바디럽 제로더스트 배기마스크 [소형/대형] (5매)', 'origin_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="0.5324250041983469-20")>, 'sales_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="0.5324250041983469-21")>}, {'link': 'https://bodyluv.kr/product/%EB%B0%94%EB%94%94%EB%9F%BD-%EC%A0%9C%EB%A1%9C-%EB%8D%94%EC%8A%A4%ED%8A%B8-%EB%A7%88%EC%8A%A4%ED%81%AC-10%EB%A7%A4/45/category/24/display/1/', 'thumbnail': 'https://bodyluv.kr/web/product/big/201710/45_shop1_270163.jpg', 'title': '바디럽 제로 더스트 마스크 (10매)', 'origin_price': <selenium.webdriver.remote.webelement.WebElement (

In [47]:
# 코드 설명 보충 위한 페이지 재로드
driver.get('https://bodyluv.kr/product/list.html?cate_no=24')

# full_list tag: .prdList
bodyluv_list = []

# 아래처럼 읽으면 됩니다. 하지만, Pagination이 있으니 끝까지 다 읽어야겠지요?
# 반복문(loop)을 사용해야 합니다.


# 반복문 적용 X
# prd_list = driver.find_element_by_class_name('prdList')
# prd_items = prd_list.find_elements_by_xpath('//li[@itemprop="itemListElement"]')
# bodyluv_list += prd_items

def bodyluv_list_structure(body_luv_item):
    thumbnail_box = body_luv_item.find_element_by_class_name('thumbnail')
    desc_box = body_luv_item.find_element_by_class_name('description')
    
    return {
        'link': thumbnail_box.find_element_by_tag_name('a').get_attribute('href'),
        'thumbnail': thumbnail_box.find_element_by_tag_name('img').get_attribute('src'),
        'title': desc_box.find_element_by_class_name('name')\
                .find_element_by_class_name('gtm-impression-click-target').text,
        'origin_price':desc_box.find_element_by_xpath('//ul/li[1]/span[1]'),
        'sales_price': desc_box.find_element_by_xpath('//ul/li[2]/span[1]'),
    }
    

# # 반복문 적용 O 
last_page_href = 'none'

next_link = None

while(True):
    prd_list = driver.find_element_by_class_name('prdList')
    prd_items = prd_list.find_elements_by_xpath('//li[@itemprop="itemListElement"]')

    # 형식에 맞게 저장.
    # 코드가 깔끔해졌지요?
    for item in prd_items:
        structured_item = bodyluv_list_structure(item)
        bodyluv_list.append(structured_item)
    
    next_page = driver.find_element_by_xpath('//*[@id="contents"]/div[3]/a[3]')
    next_link = next_page.get_attribute('href')
    
    # 형식에 맞게 정리 

    if (next_link.split('#')[-1]==last_page_href):
        break
    driver.get(next_link)

# print(len(bodyluv_list), bodyluv_list)

from pprint import pprint
pprint(bodyluv_list)





[{'link': 'https://bodyluv.kr/product/%EB%B0%94%EB%94%94%EB%9F%BD-%EB%B8%94%EB%9E%99%ED%94%84%EB%9D%BC%EC%9D%B4%EB%8D%B0%EC%9D%B4-%EC%8B%9C%EC%A6%8C2/65/category/24/display/1/',
  'origin_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="0.07663659697960035-20")>,
  'sales_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="0.07663659697960035-21")>,
  'thumbnail': 'https://bodyluv.kr/web/product/big/201902/ee08402a7626057c42c4059f2070ea39.jpg',
  'title': '바디럽 블랙프라이데이 시즌2'},
 {'link': 'https://bodyluv.kr/product/%EB%B0%94%EB%94%94%EB%9F%BD-%EB%A7%88%EC%95%BD%EB%B2%A0%EA%B0%9C/44/category/24/display/1/',
  'origin_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="0.07663659697960035-20")>,
  'sales_price': <selenium.webdriver.remote.webelement.WebElement (session="82b09a28dd94919782a24152b0bfb719", element="

#### 4. 기존 파일을 불러와서 변경사항 확인

In [53]:
# 기존 파일을 불러오기.
## 기존 파일 형식은 다음과 같습니다.
## <경쟁사이름>*<생성시간>
## 예를 들어 bodyluv을 2018년8월2일 오후 6시 30분에 분석했다 하면 파일이름은 다음과 같습니다.
## bodyluv*2018-08-02_18:30

import os

# 0. 파일 형식 입력
base_filename = 'bodyluv'
separator = '*'

# 1. 기존 파일 불러오기
file_list = []
for filename in os.listdir('.'): # 현 위치 파일 확인
    if filename.startswith(base_filename):
        file_list.append(filename)

        # 2. 기존 파일 없으면 핸들링
if not file_list:
    # 모두 신상품으로 보고, 새로운 파일을 저장합니다.
    pass
else:
    file_list.sort() # 오름차순으로 정렬합니다. 
    recent_file = file_list[-1] # 오름차순이므로 최신 파일은 리스트의 끝에 있습니다.
    
    # 기존 파일이 있으면 비교하여 새로운 항목만 신상품으로 여긴 후 메일을 보냅니다.
    # 그리고 동시에 새로운 파일을 저장합니다.
    # 




In [ ]:
5. 새로운 파일 저장
6. 변경사항이 있으면 메일을 보낸다.